In [4]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/99_STTR_phase1.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [89]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "", "Is Principal Investigator US Citizen or Permanent Resident":"",
           "Principal Investigator_Phone": "", "Principal Investigator_Company": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          "Research Institution Phone":"", "TAV Subtopics":""}
    htmlfile = open(path, 'r', encoding='cp1252')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    ### long paragraph
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential non-NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    ### basic info
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Research Topic'] = filterHTMLstr(getResearchSubTopicTitle(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    ### smb and ri info
    dic['Small Business Concern_Firm'] = filterHTMLstr(getSMBAndRIAndPIInfo(bs)[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(getSMBAndRIAndPIInfo(bs)[1])
    dic['Research Institution Name'] = filterHTMLstr(getSMBAndRIAndPIInfo(bs)[2])
    dic['Research Institution Address'] = filterHTMLstr(getSMBAndRIAndPIInfo(bs)[3])
    ### pi info
    dic['Principal Investigator_Name'] = filterHTMLstr(getSMBAndRIAndPIInfo(bs)[4])
    ### business official info
    ### others
    return dic

In [90]:
Directory_path = "../Datasets/99/sttr/phase1"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  20
../Datasets/99/sttr/phase1/STTR-99-1-01-990036.html
../Datasets/99/sttr/phase1/STTR-99-1-04-990099.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990115.html
../Datasets/99/sttr/phase1/STTR-99-1-01-990056.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990118.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990114.html
../Datasets/99/sttr/phase1/STTR-99-1-01-990109.html
../Datasets/99/sttr/phase1/STTR-99-1-04-990103.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990083.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990094.html
../Datasets/99/sttr/phase1/STTR-99-1-04-990044.html
../Datasets/99/sttr/phase1/STTR-99-1-03-990047.html
../Datasets/99/sttr/phase1/STTR-99-1-03-990088.html
../Datasets/99/sttr/phase1/STTR-99-1-04-990048.html
../Datasets/99/sttr/phase1/STTR-99-1-04-990086.html
../Datasets/99/sttr/phase1/STTR-99-1-02-990117.html
../Datasets/99/sttr/phase1/STTR-99-1-03-990108.html
../Datasets/99/sttr/phase1/STTR-99-1-03-990105.html
../Datasets/99/sttr/phase1/STTR-

# Test

In [70]:
path = "../Datasets/99/sttr/phase1/STTR-99-1-01-990056.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [60]:
### Technical Abstract and Potential Commercial Application and NASA Application
def getTechAbstract(bs):
    Ps = bs.find_all("p")
    technicalAbstract = ""
    i = 0
    while i < len(Ps):
        if Ps[i] is not None:
            line = ""
            item = Ps[i].get_text()
            if item.find("Technical Abstract (Limit 200 words)") != -1:
                while Ps[i + 1].get_text().find("Potential Commercial Application(s)(Limit 200 words)") == -1:
                    technicalAbstract = technicalAbstract.rstrip() + " " + Ps[i + 1].get_text()
                    i += 1
        i += 1
        return technicalAbstract

def getNonNASAApplication(bs):
    Ps = bs.find_all("p")
    commercialApplication = ""
    i = 0
    while i < len(Ps):
        if Ps[i] is not None:
            item = Ps[i].get_text()
            if item.find("Potential Commercial Application(s)(Limit 200 words)") != -1:
                while i + 1 < len(Ps):
                    commercialApplication = commercialApplication.rstrip() + " " + Ps[i + 1].get_text()
                    i += 1
        i += 1
    return commercialApplication

In [87]:
### Proposal Number
def getProposalNumber(bs):
    proposalNumber = ""
    pre = bs.find("pre").get_text()
    regProposalNum = re.compile(r'Proposal #: (\d+)')
    proposalNumber = re.findall(regProposalNum, pre)[0] if len(re.findall(regProposalNum, pre)) > 0 else ""
    return proposalNumber

### Phase 1 Contract Number
def getPhase1Number(bs):
    trs = bs.find_all("tr")
    Phase1Number = ""
    for tr in trs:
        if "PHASE-I CONTRACT NUMBER" in tr.get_text() or "PHASE-1 CONTRACT NUMBER" in tr.get_text() or "PHASE 1 CONTRACT NUMBER" in tr.get_text():
            Phase1Number = tr.find_all("td")[1].get_text().strip()
    return Phase1Number

### Research Topic and project title
def getResearchSubTopicTitle(bs):
    ResearchSubTopicTitle = ""
    pre = bs.find("pre").get_text()
    regResearchTopic = re.compile(r'Research Topic: (.*?)\n')
    ResearchSubTopicTitle = re.findall(regResearchTopic, pre)[0].strip() if len(re.findall(regResearchTopic, pre)[0]) else ""
    return ResearchSubTopicTitle


def getProjectTitle(bs):
    ProjectTitle = ""
    pre = bs.find("pre").get_text()
    regProjectTitle = re.compile(r'Project Title: (.*?)\n')
    ProjectTitle = re.findall(regProjectTitle, pre)[0].strip() if len(re.findall(regProjectTitle, pre)) > 0 else ""
    return ProjectTitle

In [86]:
ProjectTitle = ""
pre = bs.find("pre").get_text()
regProjectTitle = re.compile(r'Project Title: *(.*)\n\n')
re.findall(regProjectTitle, pre)

[]

In [54]:
pre = bs.find("pre").get_text()
regStateAndZip = re.compile(r'ST: *(.*?)\n')
re.findall(regStateAndZip, pre)

['OH Zip:45343 ', 'FL Zip:33431 ']

In [67]:
### Small Business and Investigator Info
def getSMBAndRIAndPIInfo(bs):
    smbName = ""
    smbAddress = ""
    riName = ""
    riAddress = ""
    piName = ""
    smbStateAndZip = ""
    riStateAndZip = ""
    smbCity = ""
    riCity = ""
    
    pre = bs.find("pre").get_text()
    regName = re.compile(r'Name: *(.*?)\n')
    names = re.findall(regName, pre)
    smbName = names[0].strip().split("  ")[0]
    piName = names[0].strip().split("  ")[-1]
    riName = names[1].strip()
    
    regAddress = re.compile(r'Address: *(.*?)\n')
    addresses = re.findall(regAddress, pre)
    smbAddress = addresses[0].strip()
    riAddress = addresses[1].strip()
    
    regCity = re.compile(r'City: *(.*?)\n')
    cities = re.findall(regCity, pre)
    smbCity = cities[0].strip()
    riCity = cities[1].strip()
    
    regStateAndZip = re.compile(r'ST: *(.*?)\n')
    statesAndZips = re.findall(regStateAndZip, pre)
    smbStateAndZip = statesAndZips[0].strip()
    riStateAndZip = statesAndZips[1].strip()
    
    
    smbAddress = smbAddress + ", " + smbCity + ", " + smbStateAndZip
    riAddress = riAddress + ", " + riCity + ", " + riStateAndZip
    return [smbName, smbAddress, riName, riAddress, piName]